<a href="https://colab.research.google.com/github/tlu-dt-nlp/POSgram-errors/blob/main/error_finder_demo_et.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sõnaliigijärjenditel põhinev veatuvastus

Siin lehel saab katsetada vealeidjat, mis otsib ebatüüpilisi sõnaliikide järgnevusi. Otsing põhineb trigrammide ehk kolmesõnaliste järjendite kasutuskontekstidel, täpsemini sellel, kui suure tõenäosusega esineb mingi trigramm lause alguses/lõpus või kindla sõnaliigi ees/järel. Statistilisele keelemudelile tuginedes eraldab programm tekstist vähetõenäolised sõnaliigijärjendid.

Vealeidja kuulub eesti keele automaatkorrektuuri tööriistakasti, vt https://koodivaramu.eesti.ee/tartunlp/corrector.


## Ettevalmistus

Tee endale rakenduse koodihoidlast koopia ja liigu kausta `posgram-errors`. Paigalda Stanza keeletöötluspakett ja käivita rakendus.

In [1]:
! git clone https://github.com/tlu-dt-nlp/posgram-errors.git

Cloning into 'posgram-errors'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 68 (delta 33), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (68/68), 373.88 KiB | 4.20 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [3]:
%cd posgram-errors

/content/posgram-errors


In [3]:
! pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 17.6 MB/s eta 0:00:00


In [4]:
from posgram_finder import PosgramFinder
p = PosgramFinder()
# Vaikimisi peetakse trigrammi konteksti vähetõenäoliseks, kui selle osakaal keelemudelis on alla 5%.
# Seda seadet saab muuta argumendi "lower_percentage_limit" abil, nt:
#p = PosgramFinder(lower_percentage_limit=2.5)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: et (Estonian):
| Processor | Package      |
----------------------------
| tokenize  | edt          |
| pos       | edt_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


## Veatuvastus eelnevalt parandamata tekstides

Tööriista võib kasutada nii eelnevalt parandamata tekstide kui ka automaatse veaparanduse väljundi analüüsimiseks. Kahes järgnevas näites analüüsime eesti keelt teise keelena õppijate algupäraseid lauseid.

Kasutajale kuvatakse lause esitus sõnaliigimärgendite jadana (vt märgendite seletusi https://github.com/tlu-dt-nlp/POSgram-errors) ning tuuakse välja ebatüüpiline sõnanelik (lause sees) või -kolmik (lause alguses/lõpus). Tähistatakse järjendi asukoht lauses sõnede järjekorranumbrite abil (esimesele sõnele vastab 0, teisele sõnele 1 jne), vähetõenäolise konteksti tüüp (ees/järelkontekst) ja keelemudelil põhinev osakaal vastava trigrammi kontekstide hulgas. Kirjavahemärke analüüsis ei arvestata, kuid kuvatakse nende asukoht sõnajärjendis.

### 1. Sõnavalikuviga

Omadussõna *kiire* asemel on kasutatud määrsõna *kiiresti*. Vigasele sõnale osutab kolm ebatüüpilist sõnaliigijärjendit. Üks neist hõlmab harvaesineva eeskontekstiga ja kaks harvaesineva järelkontekstiga trigrammi.

In [5]:
result = p.posgram_errors("See on väga ilus, kiiresti ja mugav punane auto.")

^PVDAZDJAASZ$


In [ ]:
from pprint import pprint
pprint(result, sort_dicts=False)

[{'sentence': 'See on väga ilus, kiiresti ja mugav punane auto.',
  'sentence_posgram': '^PVDAZDJAASZ$',
  'error_candidates': [{'value': 'on väga ilus , kiiresti',
                        'posgram': 'VDAZD',
                        'start_token': 1,
                        'end_token': 5,
                        'trigram': 'VDA',
                        'type': 'post',
                        'context': 'D',
                        'percent': 3.085772778093205},
                       {'value': 'ilus , kiiresti ja mugav',
                        'posgram': 'AZDJA',
                        'start_token': 3,
                        'end_token': 7,
                        'trigram': 'DJA',
                        'type': 'pre',
                        'context': 'A',
                        'percent': 1.9070039340843379},
                       {'value': 'kiiresti ja mugav punane',
                        'posgram': 'DJAA',
                        'start_token': 5,
                      

### 2. Puuduv tegusõna

Kõrvallausest *nagu kodumasin katki* on puudu tegusõna *olema*. Sellele viitab kolm vähetõenäolist sõnajärjendit. Trigrammil tegusõna-sidesõna-nimisõna (VJS) on nii ebatüüpiline eeskontekst (sidesõna) kui ka järelkontekst (määrsõna). Sidesõna-nimisõna-määrsõna (JSD) esineb harva lause lõpus.

In [ ]:
sentence = "Eile koristasin vannituba ja näen nagu kodumasin katki."
result = p.posgram_errors(sentence)

^DVSJVJSDZ$


In [ ]:
pprint(result[0]["error_candidates"], sort_dicts=False)

[{'value': 'ja näen nagu kodumasin',
  'posgram': 'JVJS',
  'start_token': 3,
  'end_token': 6,
  'trigram': 'VJS',
  'type': 'pre',
  'context': 'J',
  'percent': 4.7853188712363295},
 {'value': 'näen nagu kodumasin katki',
  'posgram': 'VJSD',
  'start_token': 4,
  'end_token': 7,
  'trigram': 'VJS',
  'type': 'post',
  'context': 'D',
  'percent': 4.402904046086682},
 {'value': 'nagu kodumasin katki .',
  'posgram': 'JSDZ$',
  'start_token': 5,
  'end_token': 9,
  'trigram': 'JSD',
  'type': 'post',
  'context': '$',
  'percent': 4.90649701176017}]


## Veatuvastus automaatparanduse väljundis

Järgnevalt analüüsime kahte lauset, mida on töödeldud eesti keele grammatikavigade paranduses parimaid tulemusi saavutanud korrektuurimudeliga (NELB).

Esimest lauset ei ole veaparandaja muutnud. Teist lauset on oluliselt korrigeeritud, algselt oli see: *Seal palju kohvikud, muuseumusid kino*.

In [ ]:
text = "Vitebskis me kuulasime kontserti. Seal on palju kohvikuid, muuseume, kinosid."
result = p.posgram_errors(text)

^SPVSZ$
^DVDSZSZSZ$


### 1. Sõnajärjeviga

Esimese lause alguses on järjend nimisõna-asesõna-tegusõna (SPV), mis ei ühti V2-sõnajärje reegliga. Kirjalikus tekstis oleks soovitatav kasutada sõnajärge *Vitebskis kuulasime me*.

### 2. Puuduv sidesõna

Määrsõnale ei järgne tüüpiliselt kolm järjestikust nimisõna. Ehkki tegemist ei ole grammatikaveaga, muudaks lause ladusamaks see, kui lisada loetelusse sõnade *muuseume* ja *kinosid* vahele sidesõna *ja*.

In [ ]:
pprint(result, sort_dicts=False)

[{'sentence': 'Vitebskis me kuulasime kontserti.',
  'sentence_posgram': '^SPVSZ$',
  'error_candidates': [{'value': 'Vitebskis me kuulasime',
                        'posgram': 'SPV',
                        'start_token': 0,
                        'end_token': 2,
                        'trigram': 'SPV',
                        'type': 'pre',
                        'context': '^',
                        'percent': 4.862718076808973}]},
 {'sentence': 'Seal on palju kohvikuid, muuseume, kinosid.',
  'sentence_posgram': '^DVDSZSZSZ$',
  'error_candidates': [{'value': 'palju kohvikuid , muuseume , kinosid',
                        'posgram': 'DSZSZS',
                        'start_token': 2,
                        'end_token': 7,
                        'trigram': 'SSS',
                        'type': 'pre',
                        'context': 'D',
                        'percent': 3.9641244155294872}]}]


Proovi analüüsida ka oma lauset või teksti.